---
# Deliverable 2 — Feature Engineering & Classical ML
**Task:** Feature Engineering + Machine Learning classifiers (LR, SVM, DT, RF, XGBoost)  
**Subject Data used:** WESAD — S2.pkl (Chest + Wrist)  
---


## 1. Objective (Week-1 Deliverable)

Implement Approach-1: Feature engineering + classical ML classifiers.

Deliverables:
- Feature extraction from multimodal sensors (chest + wrist)
- Train classifiers: Logistic Regression, SVM, Decision Tree, Random Forest, XGBoost
- Evaluate models (Accuracy, Precision, Recall, F1, Confusion Matrix, ROC-AUC)
- Save `features.csv`, `code.ipynb`, and a small report (report.pdf)


## 2. Dataset Summary (Short Recap)

- **Dataset name:** WESAD — Wearable Stress and Affect Detection  
- **Source / Link:** UCI ML Repository — https://archive.ics.uci.edu/dataset/465/wesad+wearable+stress+and+affect+detection  
- **Subject file used:** `S2/S2.pkl` (example subject; same code works for others)  
- **Sensors used (this deliverable):**
  - Chest: ACC_x, ACC_y, ACC_z, ECG, EMG, EDA, Resp, Temp
  - Wrist: ACC_x, ACC_y, ACC_z, BVP, EDA, TEMP
- **Sampling rates:** chest ≈ 700 Hz; wrist ≈ 32 Hz (documented in dataset)
- **Problem type:** Multi-class classification (states 0 Baseline, 1 Stress, 2 Amusement). Other labels (3–7) exist in the raw file and will be ignored for the classification task.


## 3. Notebook Outline

1. Load WESAD S2.pkl and inspect structure  
2. Build chest/wrist arrays and timestamps  
3. Segment signals into fixed windows (10-second windows by default)  
4. Extract features (time-domain, frequency-domain, physiological) from each window for all sensors  
5. Label windows via majority vote of chest labels (keep only classes 0,1,2)  
6. Build `features.csv` (X) and target vector (y)  
7. Train-test split (stratified), scale where needed  
8. Train classifiers (LR, SVM, DT, RF, XGBoost)  
9. Evaluate metrics, plot confusion matrices, compute ROC-AUC (multiclass OVR)  
10. Save results and show feature importances


In [1]:
# 4. Import libraries 
import os
import pickle
import math
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks
from scipy.fft import fft
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")


## 5. Load WESAD S2.pkl and inspect structure


In [2]:
# 5. Load data
data_path = r"D:\Semester 7\ML\Project\WESAD DS\WESAD\S2\S2.pkl" 

with open(data_path, 'rb') as f:
    data = pickle.load(f, encoding='latin1')

# quick inspection
print("Top-level keys:", list(data.keys()))
# signals dictionary keys
print("Signal groups:", list(data['signal'].keys()))
print("Chest keys:", list(data['signal']['chest'].keys()))
print("Wrist keys:", list(data['signal']['wrist'].keys()))
print("Labels shape:", np.array(data['label']).shape)


Top-level keys: ['signal', 'label', 'subject']
Signal groups: ['chest', 'wrist']
Chest keys: ['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']
Wrist keys: ['ACC', 'BVP', 'EDA', 'TEMP']
Labels shape: (4255300,)


**Notes**
- `data['signal']['chest']` and `data['signal']['wrist']` are dictionaries mapping sensor names to numpy arrays.
- `data['label']` contains time-aligned labels (length matches chest signals).


In [3]:
# extract references to chest, wrist, labels
chest = data['signal']['chest']
wrist = data['signal']['wrist']
labels = np.array(data['label'])  # 1D array; length ~ len(chest signals)


## 6. Build DataFrames / arrays for sensors (quick consistency checks)
- We'll not convert everything to a single massive DataFrame (very large) — instead we'll operate on numpy arrays and create window-level features.
- Confirm shapes and sampling rates (we use documented sampling rates below).


In [4]:
# print shapes for chest sensors
for k, v in chest.items():
    print("chest.", k, "shape:", np.array(v).shape)
print()
for k, v in wrist.items():
    print("wrist.", k, "shape:", np.array(v).shape)


chest. ACC shape: (4255300, 3)
chest. ECG shape: (4255300, 1)
chest. EMG shape: (4255300, 1)
chest. EDA shape: (4255300, 1)
chest. Temp shape: (4255300, 1)
chest. Resp shape: (4255300, 1)

wrist. ACC shape: (194528, 3)
wrist. BVP shape: (389056, 1)
wrist. EDA shape: (24316, 1)
wrist. TEMP shape: (24316, 1)


## 7. Windowing strategy
- Window duration: 10 seconds (configurable).  
- chest_freq = 700 Hz → window_size_chest = 700 * 10 = 7000 samples.  
- wrist_freq = 32 Hz → window_size_wrist = 32 * 10 = 320 samples.  
- Number of windows used: `min(floor(len(chest_signal)/window_size_chest), floor(len(wrist_signal)/window_size_wrist))` to ensure full data exists for both chest & wrist for each window index.


In [5]:
# 7. Window parameters
chest_fs = 700
wrist_fs = 32
window_sec = 10
w_chest = int(chest_fs * window_sec)
w_wrist = int(wrist_fs * window_sec)

# compute number of windows available
num_windows_chest = len(chest['ACC']) // w_chest
# choose wrist base on 'ACC' or 'BVP' whichever exists
num_windows_wrist = len(wrist['ACC']) // w_wrist
num_windows = min(num_windows_chest, num_windows_wrist)

print(f"Window size (samples): chest={w_chest}, wrist={w_wrist}")
print("Available windows: chest:", num_windows_chest, "wrist:", num_windows_wrist, "using:", num_windows)


Window size (samples): chest=7000, wrist=320
Available windows: chest: 607 wrist: 607 using: 607


## 8. Helper functions — feature extraction
We implement:
- time-domain features
- frequency-domain features (FFT based)
- sensor-specific physiological features:
  - ACC: vector magnitude and SMA
  - EDA: peak counts, mean, tonic
  - ECG: simple R-peak approach for RR intervals → HR and RMSSD (approximate)
  - EMG: RMS and zero-crossing rate
  - Resp & Temp & BVP: statistical + spectral


In [6]:
# 8. Feature functions

def time_features(x):
    """Return dict of common time-domain features for 1D numpy array x."""
    x = np.asarray(x).astype(float)
    feats = {}
    feats['mean'] = np.mean(x)
    feats['median'] = np.median(x)
    feats['std'] = np.std(x)
    feats['var'] = np.var(x)
    feats['min'] = np.min(x)
    feats['max'] = np.max(x)
    feats['25perc'] = np.percentile(x, 25)
    feats['75perc'] = np.percentile(x, 75)
    feats['skew'] = skew(x) if x.size > 2 else 0.0
    feats['kurtosis'] = kurtosis(x) if x.size > 3 else 0.0
    feats['rms'] = np.sqrt(np.mean(x**2))
    return feats

def freq_features(x, fs):
    """Return basic frequency-domain features for 1D array x sampled at fs."""
    x = np.asarray(x).astype(float)
    n = len(x)
    if n < 2:
        return {'dominant_freq':0.0,'spectral_energy':0.0,'power_low':0.0,'power_high':0.0}
    X = np.abs(fft(x))
    X = X[:n//2]  # positive freqs
    freqs = np.fft.fftfreq(n, d=1.0/fs)[:n//2]
    # avoid nan errors
    if np.all(X == 0):
        dominant = 0.0
    else:
        dominant = float(freqs[np.nanargmax(X)])
    spectral_energy = float(np.sum(X**2))
    # HRV/resp bands: low (0.04-0.15), high (0.15-0.4) — meaningful for HR/resp
    low_mask = (freqs >= 0.04) & (freqs <= 0.15)
    high_mask = (freqs >= 0.15) & (freqs <= 0.4)
    power_low = float(np.sum(X[low_mask]**2)) if low_mask.any() else 0.0
    power_high = float(np.sum(X[high_mask]**2)) if high_mask.any() else 0.0
    return {'dominant_freq':dominant,'spectral_energy':spectral_energy,'power_low':power_low,'power_high':power_high}

def eda_features(x):
    x = np.asarray(x).astype(float).flatten()
    feats = {}
    # detect peaks (simple) — threshold small positive to ignore tiny noise
    try:
        peaks, _ = find_peaks(x, height=np.std(x)*0.2 if np.std(x)>0 else 0.01)
        feats['eda_num_peaks'] = len(peaks)
    except Exception:
        feats['eda_num_peaks'] = 0
    feats['eda_mean'] = np.mean(x)
    feats['eda_tonic'] = np.min(x)  # tonic as minimum in the window
    return feats

def acc_physio_features(x3):
    # x3: Nx3 accelerometer window array
    x3 = np.asarray(x3).astype(float)
    # vector magnitude
    vm = np.sqrt((x3**2).sum(axis=1))
    feats = {}
    feats['acc_vm_mean'] = np.mean(vm)
    feats['acc_vm_std'] = np.std(vm)
    feats['acc_sma'] = np.sum(np.abs(vm)) / len(vm) if len(vm)>0 else 0.0
    feats['acc_vm_rms'] = np.sqrt(np.mean(vm**2)) if len(vm)>0 else 0.0
    return feats

def emg_features(x, fs):
    x = np.asarray(x).astype(float)
    feats = time_features(x)
    # zero crossing rate
    zc = np.sum(np.abs(np.diff(np.sign(x))))/ (2*len(x)) if len(x)>1 else 0.0
    feats['emg_zcr'] = zc
    feats['emg_rms'] = np.sqrt(np.mean(x**2)) if len(x)>0 else 0.0
    # spectral energy for EMG
    ff = freq_features(x, fs)
    feats.update({'emg_spec_energy': ff['spectral_energy'], 'emg_dom_freq': ff['dominant_freq']})
    return feats

def ecg_hrv_features(x, fs):
    """
    A simple R-peak like detection using find_peaks on ECG.
    This is approximate — for production use a specialized ECG R-peak detector (e.g., Pan-Tompkins).
    Returns heart rate and simple HRV metric (RMSSD).
    """
    x = np.asarray(x).astype(float).flatten()
    feats = {}
    if len(x) < 10:
        feats['ecg_hr_mean'] = 0.0
        feats['ecg_rmssd'] = 0.0
        return feats
    # basic peak detection — require peaks bigger than mean+std
    try:
        height_thr = np.mean(x) + 0.5*np.std(x)
        peaks, _ = find_peaks(x, height=height_thr, distance=int(0.4*fs))  # at least 0.4s between peaks
    except Exception:
        peaks = np.array([])
    # compute RR intervals (seconds)
    if len(peaks) >= 2:
        rr = np.diff(peaks) / float(fs)  # seconds
        hr = 60.0 / np.mean(rr) if np.mean(rr)>0 else 0.0
        # RMSSD
        diffs = np.diff(rr)
        rmssd = np.sqrt(np.mean(diffs**2)) if diffs.size>0 else 0.0
    else:
        hr = 0.0
        rmssd = 0.0
    feats['ecg_hr_mean'] = float(hr)
    feats['ecg_rmssd'] = float(rmssd)
    return feats

def simple_spectral_entropy(x, fs):
    """(Optional) approximate spectral entropy using FFT power distribution."""
    x = np.asarray(x).astype(float)
    n = len(x)
    if n < 2:
        return 0.0
    X = np.abs(fft(x))[:n//2]
    P = X**2
    if P.sum() == 0:
        return 0.0
    P_norm = P / P.sum()
    # entropy = -sum(P * log(P))
    entropy = -np.sum(P_norm * np.log2(P_norm + 1e-12))
    return float(entropy)


## 9. Construct window-level features for *all* selected sensors
- For each window index `i` from 0 to `num_windows-1`, extract the chest and wrist segments.
- Compute sensor-specific features and combine into a single feature vector per window.
- Label each window by majority chest label inside that window.
- Keep only windows whose majority label ∈ {0,1,2}.


In [7]:
# 9. Build features across windows (this may take a few minutes)
features = []
window_labels = []

for i in range(num_windows):
    # chest indices (samples)
    c_start = i * w_chest
    c_end = c_start + w_chest
    # wrist indices
    w_start = i * w_wrist
    w_end = w_start + w_wrist

    # chest segments
    acc_ch_seg = chest['ACC'][c_start:c_end]         # shape (w_chest, 3)
    ecg_ch_seg = chest['ECG'][c_start:c_end].flatten()
    emg_ch_seg = chest['EMG'][c_start:c_end].flatten()
    eda_ch_seg = chest['EDA'][c_start:c_end].flatten()
    resp_ch_seg = chest['Resp'][c_start:c_end].flatten()
    temp_ch_seg = chest['Temp'][c_start:c_end].flatten()

    # wrist segments (slice to available)
    acc_wr_seg = wrist['ACC'][w_start:w_end] if 'ACC' in wrist else np.zeros((w_wrist,3))
    bvp_wr_seg = wrist['BVP'][w_start:w_end].flatten() if 'BVP' in wrist else np.zeros(w_wrist)
    eda_wr_seg = wrist['EDA'][w_start:w_end].flatten() if 'EDA' in wrist else np.zeros(w_wrist)
    temp_wr_seg = wrist['TEMP'][w_start:w_end].flatten() if 'TEMP' in wrist else np.zeros(w_wrist)

    # start building per-window features
    win_feats = {}

    # chest ACC features (vector magnitude + time + freq)
    win_feats.update({f"ch_{k}": v for k, v in acc_physio_features(acc_ch_seg).items()})
    # also time-domain per axis
    for axis_idx, axis_name in enumerate(['acc_x','acc_y','acc_z']):
        tfeats = time_features(acc_ch_seg[:, axis_idx])
        tfeats = {f"ch_{axis_name}_{k}": v for k, v in tfeats.items()}
        win_feats.update(tfeats)
        ffeats = freq_features(acc_ch_seg[:, axis_idx], chest_fs)
        ffeats = {f"ch_{axis_name}_{k}": v for k, v in ffeats.items()}
        win_feats.update(ffeats)

    # chest EDA
    win_feats.update({f"ch_{k}": v for k, v in eda_features(eda_ch_seg).items()})
    win_feats['ch_eda_spec_entropy'] = simple_spectral_entropy(eda_ch_seg, chest_fs)

    # chest ECG HRV features
    win_feats.update({f"ch_{k}": v for k, v in ecg_hrv_features(ecg_ch_seg, chest_fs).items()})
    # chest ECG spectral/time features for waveform
    win_feats.update({f"ch_ecg_{k}": v for k, v in time_features(ecg_ch_seg).items()})
    win_feats.update({f"ch_ecg_{k}": v for k, v in freq_features(ecg_ch_seg, chest_fs).items()})

    # chest EMG features
    win_feats.update({f"ch_emg_{k}": v for k, v in emg_features(emg_ch_seg, chest_fs).items()})

    # chest Resp features
    win_feats.update({f"ch_resp_{k}": v for k, v in time_features(resp_ch_seg).items()})
    win_feats.update({'ch_resp_spec_entropy': simple_spectral_entropy(resp_ch_seg, chest_fs)})
    win_feats.update({f"ch_resp_{k}": v for k, v in freq_features(resp_ch_seg, chest_fs).items()})

    # chest Temp features (slow changing)
    win_feats.update({f"ch_temp_{k}": v for k, v in time_features(temp_ch_seg).items()})

    # wrist ACC features
    try:
        win_feats.update({f"wr_{k}": v for k, v in acc_physio_features(acc_wr_seg).items()})
    except Exception:
        pass
    for axis_idx, axis_name in enumerate(['acc_x','acc_y','acc_z']):
        tfeats = time_features(acc_wr_seg[:, axis_idx])
        win_feats.update({f"wr_{axis_name}_{k}": v for k, v in tfeats.items()})
        ffeats = freq_features(acc_wr_seg[:, axis_idx], wrist_fs)
        win_feats.update({f"wr_{axis_name}_{k}": v for k, v in ffeats.items()})

    # wrist BVP
    win_feats.update({f"wr_bvp_{k}": v for k, v in time_features(bvp_wr_seg).items()})
    win_feats.update({f"wr_bvp_{k}": v for k, v in freq_features(bvp_wr_seg, wrist_fs).items()})

    # wrist EDA
    win_feats.update({f"wr_eda_{k}": v for k, v in eda_features(eda_wr_seg).items()})
    win_feats.update({'wr_eda_spec_entropy': simple_spectral_entropy(eda_wr_seg, wrist_fs)})

    # wrist temp
    win_feats.update({f"wr_temp_{k}": v for k, v in time_features(temp_wr_seg).items()})

    # label majority vote from chest labels
    lab_window = labels[c_start:c_end]
    if len(lab_window) == 0:
        majority_label = -1
    else:
        majority_label = int(np.bincount(lab_window).argmax())

    features.append(win_feats)
    window_labels.append(majority_label)

# convert to DataFrame
X_all = pd.DataFrame(features)
y_all = np.array(window_labels)

print("Constructed feature matrix shape:", X_all.shape)
print("Raw label distribution:", np.unique(y_all, return_counts=True))


d:\Semester 7\ML\Project\.venv\Lib\site-packages\numpy\_core\_methods.py:222: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
d:\Semester 7\ML\Project\.venv\Lib\site-packages\numpy\_core\_methods.py:180: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
d:\Semester 7\ML\Project\.venv\Lib\site-packages\numpy\_core\_methods.py:214: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
d:\Semester 7\ML\Project\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\Semester 7\ML\Project\.venv\Lib\site-packages\numpy\_core\_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: zero-size array to reduction operation minimum which has no identity

## 10. Filter windows to include only target classes (0,1,2) and handle missing values
- Remove windows whose majority label is not in {0,1,2}
- Remove features with all-NaN or constant values
- Fill or drop remaining NaNs conservatively


In [8]:
# 10. Clean features and labels
valid_mask = np.isin(y_all, [0,1,2])
X = X_all[valid_mask].reset_index(drop=True)
y = y_all[valid_mask]

print("After filtering to classes 0/1/2 -> X shape:", X.shape, "y shape:", y.shape)
# drop columns that are all NaN or constant
nunique = X.nunique(dropna=True)
drop_cols = list(nunique[nunique <= 1].index)
X = X.drop(columns=drop_cols)
print("Dropped constant columns:", drop_cols)

# Fill remaining NaNs with column median (robust)
X = X.fillna(X.median())


NameError: name 'y_all' is not defined

## 11. Save feature matrix (optional but required in submission)
- Save `features.csv` for submission/analysis


In [9]:
# 11. Save features and labels to CSV
out_dir = "./deliverable2_outputs"
os.makedirs(out_dir, exist_ok=True)
features_csv = os.path.join(out_dir, "features.csv")
X.to_csv(features_csv, index=False)
pd.DataFrame({'label': y}).to_csv(os.path.join(out_dir, "labels.csv"), index=False)
print("Saved features to:", features_csv)


NameError: name 'X' is not defined

## 12. Train/Test split + scaling
- Use stratified split to preserve class balance
- Use StandardScaler for LR and SVM


In [10]:
# 12. Train/test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Train/test sizes:", X_train.shape, X_test.shape)

# Scaling (fit on train only)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


NameError: name 'X' is not defined

## 13. Models to train
We will train and evaluate:
- Logistic Regression (use scaled features)
- SVM (scaled)
- Decision Tree (unscaled)
- Random Forest (unscaled)
- XGBoost (unscaled)

We will compute accuracy, precision, recall, F1 and multiclass ROC-AUC (one-vs-rest).


In [11]:
# 13. Define models
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial'),
    'SVM': SVC(probability=True),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
}


In [13]:
# 14. Training and evaluation loop
from sklearn.preprocessing import label_binarize

classes = np.unique(y_train)
y_test_bin = label_binarize(y_test, classes=classes)

results = {}

for name, model in models.items():
    print(f"\n===== MODEL: {name} =====")
    if name in ['LogisticRegression', 'SVM']:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_proba = model.predict_proba(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        # tree-based .predict_proba probably available
        try:
            y_proba = model.predict_proba(X_test)
        except Exception:
            y_proba = None

    # classification report
    print(classification_report(y_test, y_pred, digits=4, zero_division=0))
    # confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=classes)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix - {name}')
    plt.show()

    # accuracy
    acc = accuracy_score(y_test, y_pred)
    # ROC-AUC (multiclass OVR)
    try:
        if y_proba is not None and y_proba.shape[1] == len(classes):
            y_test_binarized = label_binarize(y_test, classes=classes)
            roc_auc = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')
        else:
            roc_auc = float('nan')
    except Exception:
        roc_auc = float('nan')

    print(f"Accuracy: {acc:.4f}  |  ROC-AUC (ovr): {roc_auc:.4f}")
    results[name] = {'accuracy': acc, 'roc_auc': roc_auc, 'model': model}


NameError: name 'y_train' is not defined